# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

In [3]:
### Load the Data from Pickle File without having to run things from the beginning
import tensorflow as tf
from tensorflow.python.ops.variables import Variable
import cv2
import numpy as np
import scipy.ndimage
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import confusion_matrix
import pickle
import os
import math
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import Image
%matplotlib inline

# Reload the data
def load_data(filename):
    pickle_file = filename + '.pickle'
    with open(pickle_file, 'rb') as f:
        pickle_data = pickle.load(f)
        train_features = pickle_data['train_dataset']
        train_labels = pickle_data['train_labels']
        valid_features = pickle_data['valid_dataset']
        valid_labels = pickle_data['valid_labels']
        test_features = pickle_data['test_dataset']
        test_labels = pickle_data['test_labels']
        train_dict_features = pickle_data['train_features_accuracy']
        train_dict_labels = pickle_data['train_labels_accuracy']
        del pickle_data  # Free up memory
    print('Data and modules loaded')
    return train_features, train_labels, valid_features, valid_labels, test_features, test_labels, train_dict_features, train_dict_labels

In [4]:
def print_images(data, indices=[]):
    if len(data.shape) <= 3:
        fig = plt.figure()
        ax = fig.add_subplot(111)
        if (data.shape[-1] == 1):
            data = data[:, :, 0]
            ax.imshow(data, cmap='gray')
        else:
            ax.imshow(data)
        if len(indices) > 0:
            print(indices[0])
        plt.show()
    elif len(data.shape) >= 4:
        for d in range(len(data)):
            fig = plt.figure()
            ax = fig.add_subplot(111)
            if (data[d].shape[-1] == 1):
                ax.imshow(data[d, :, :, 0], cmap='gray')
            else:
                ax.imshow(data[d])
            if len(indices) > 0:
                print(indices[d])
            plt.show()
            
def print_class(features, labels, class_no, image_idx):
    if len(labels.shape) > 1:
        if len(labels) == 1:
            indices_by_class = np.where(np.argmax(labels) == class_no)
        else:
            indices_by_class = np.where(np.argmax(labels, axis=1) == class_no)[0]
    else:
        if len(labels) == 1:
            indices_by_class = np.where(labels == class_no)
        else:
            indices_by_class = np.where(labels == class_no)[0]
            
    features_by_class = features[indices_by_class]
    images = features_by_class[image_idx]
    print_images(images, image_idx)
    

In [5]:
def conv_layer(input_layer, filter_size, num_input_channels, num_filters, stride=1, padding='SAME',
               relu=False, pooling=False, pool_size=2, drop=False, keep_prob=0.5, name=None):
    
    shape = [filter_size, filter_size, num_input_channels, num_filters]

    # Create filter weights with the given shape.
    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    # print(weights.get_shape())

    # Create new biases, one for each filter.
    biases = tf.Variable(tf.zeros(num_filters))

    # Create the convolution operation
    conv_layer = tf.nn.conv2d(input=input_layer,
                             filter=weights,
                             strides=[1, stride, stride, 1],
                             padding=padding)

    # Add the biases to the convolution output
    conv_layer = tf.nn.bias_add(conv_layer, biases)
    
    if relu:
        # Create the ReLu activation function
        conv_layer = tf.nn.relu(conv_layer)
        
    if drop:
        # Use Dropout to prevent overfitting
        conv_layer = tf.nn.dropout(conv_layer, keep_prob)
        
    if pooling == 'max':
        # Create max-pooling
        conv_layer = tf.nn.max_pool(value=conv_layer,
                               ksize=[1, pool_size, pool_size, 1],
                               strides=[1, pool_size, pool_size, 1],
                               padding=padding)
    if pooling == 'average':
        # Create average-pooling
        conv_layer = tf.nn.avg_pool(value=conv_layer,
                               ksize=[1, pool_size, pool_size, 1],
                               strides=[1, pool_size, pool_size, 1],
                               padding=padding)

    print(conv_layer.get_shape())
    
    return conv_layer

def flatten_layer(input_layer):
    layer_shape = input_layer.get_shape()

    # The number of features is: img_height * img_width * num_channels
    num_features = layer_shape[1:4].num_elements()

    # Reshape the layer to [num_images, num_features]
    flat_layer = tf.reshape(input_layer, [-1, num_features])
    print(flat_layer.get_shape())
    
    return (flat_layer)
    
def fc_layer(input_layer, num_features, fc_size, relu=False, drop=False, keep_prob=0.5):
    ## Create the 1st Fully Connected layer
    # Create new weights and biases.
    shape=[num_features, fc_size]

    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    biases = tf.Variable(tf.zeros(fc_size))

    # Calculate the layer as the matrix multiplication of
    # the input and weights, and then add the bias-values.
    fc_layer = tf.nn.bias_add(tf.matmul(input_layer, weights), biases)
    
    if relu:
        fc_layer = tf.nn.relu(fc_layer)
    
    if drop:
        fc_layer = tf.nn.dropout(fc_layer, keep_prob)
    print(fc_layer.get_shape())
    
    return fc_layer


In [6]:
# Load the data from pickle file
filename = 'traffic-signs-rgb'
# filename = 'traffic-signs-grayscale'
# filename = 'traffic-signs-yuv'
train_features, train_labels, valid_features, valid_labels, test_features, test_labels, train_dict_features, train_dict_labels = load_data(filename)

image_shape = train_features[0].shape
n_classes = train_labels.shape[1]

print(n_classes)
print(image_shape)
print(train_features.shape, train_labels.shape)
print(valid_features.shape, valid_labels.shape)
print(test_features.shape, test_labels.shape)
print(train_dict_features.shape, train_dict_labels.shape)

Data and modules loaded
43
(32, 32, 3)
(116337, 32, 32, 3) (116337, 43)
(1290, 32, 32, 3) (1290, 43)
(12630, 32, 32, 3) (12630, 43)
(1290, 32, 32, 3) (1290, 43)


In [10]:
### Create a ConvNet
from tensorflow.contrib.layers import flatten

# Create the input data placeholders
features = tf.placeholder(tf.float32, shape=[None, *train_features.shape[1:]])
labels = tf.placeholder(tf.float32, shape=[None, train_labels.shape[-1]])

filter_size = 5
output_size = 6

# Conv Layer 1
conv_layer_1 = conv_layer(features, filter_size, train_features.shape[-1], output_size, 
                          padding='VALID', relu=True)

# Conv Layer 2
conv_layer_2 = conv_layer(conv_layer_1, filter_size, output_size, 16, 
                          padding='VALID', relu=True)

# Conv Layer 3
conv_layer_3 = conv_layer(conv_layer_2, filter_size, 16, 32, 
                          padding='VALID', pooling='max', relu=True, drop=True)

# Flatten layer
flat_layer = flatten(conv_layer_3)
print(flat_layer.get_shape())

# Fully Connected Layer 1
num_features = int(flat_layer.get_shape()[-1])
fc1_size = 120
fc_layer_1 = fc_layer(flat_layer, num_features, fc1_size, relu=True)

# Fully Connected Layer 2
fc2_size = 84
fc_layer_2 = fc_layer(fc_layer_1, fc1_size, fc2_size, relu=True)

# Output Layer
output_layer = fc_layer(fc_layer_2, fc2_size, train_labels.shape[-1])

(?, 28, 28, 6)
(?, 24, 24, 16)
(?, 10, 10, 32)
(?, 3200)
(?, 120)
(?, 84)
(?, 43)


In [ ]:
## Creating the Softmax predictions, loss, and optimizer functions
prediction = tf.nn.softmax(output_layer)
y_pred_cls = tf.argmax(prediction, 1)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_layer, labels))

learning_rate = 0.001

# Gradient Descent
# optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# Test model
correct_prediction = tf.equal(y_pred_cls, tf.argmax(labels, 1))

# Calculate accuracy
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data, batch_size):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = X_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
        accuracy = sess.run(accuracy_operation, feed_dict={features: batch_x, labels: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

print('Optimization and Evaluation Functions Created')

In [12]:
# Import shuffle from sklearn to shuffle training data after every epoch
from sklearn.utils import shuffle

# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()
init = tf.initialize_all_variables()

# The file path to save the data
save_file = 'model_rgb.ckpt'

valid_feed_dict = {features: valid_features, labels: valid_labels}
test_feed_dict = {features: test_features, labels: test_labels}
train_feed_dict = {features: train_dict_features, labels: train_dict_labels}

def train_model(train_features, train_labels, run_count, epochs, batch_size, save_file):
    # Launch the graph
    with tf.Session() as sess:
        if run_count == 0:
            print('Initiating the Model')
            sess.run(init)
        else:
            print('Restoring a Saved Model')
            saver.restore(sess, save_file)
        
        batch_count = int(math.ceil(len(train_features)/batch_size))
        
        # Training cycle
        for epoch_i in range(epochs):
            # Shuffle the training data
            train_features, train_labels = shuffle(train_features, train_labels)
            # Progress bar
            batches_pbar = tqdm(range(batch_count), desc='Epoch {:>2}/{}'.format(epoch_i+1, epochs), unit='batches')

            # The training cycle
            for batch_i in batches_pbar:
                # Get a batch of training features and labels
                batch_start = batch_i*batch_size
                if batch_start >= len(train_features):
                    break
                batch_end = min(batch_start + batch_size, len(train_features))
                batch_features = train_features[batch_start:batch_end]
                batch_labels = train_labels[batch_start:batch_end]

                # Run optimizer and get loss
                _, l = sess.run(
                    [optimizer, loss],
                    feed_dict={features: batch_features, labels: batch_labels})

            # Check accuracy against Validation data
            validation_accuracy = evaluate(valid_features, valid_labels, batch_size)
            training_accuracy = evaluate(train_dict_features, train_dict_labels, batch_size)
                  
            # Display logs per epoch step
            print("Epoch:", '%04d' % (epoch_i+1), "loss=", "{:.9f}".format(l)) 
            print("Validation Accuracy=", "{:.9f}".format(validation_accuracy), 
                  "Training Accuracy=", "{:.9f}".format(training_accuracy))

        print("Optimization Finished!") 

        # Save the model
        saver.save(sess, save_file)
        print('Model Saved to ' + save_file)

        # Check accuracy against Validation data
        validation_accuracy = sess.run(accuracy_operation, feed_dict=valid_feed_dict)
        print('Validation accuracy at {}'.format(validation_accuracy))

        # Check accuracy against Validation data
        test_accuracy = sess.run(accuracy_operation, feed_dict=test_feed_dict)
        print('Test accuracy at {}'.format(test_accuracy))

        exam_size = 500
        y_labels = tf.placeholder(tf.float32, shape=[None, n_classes])
        y_true_cls = tf.argmax(y_labels, 1)
        cls_true, cls_pred = sess.run([y_true_cls, y_pred_cls], feed_dict={features: test_features[0:exam_size], 
                                                                       y_labels: test_labels[0:exam_size]})
        print(cls_true)
        print(cls_pred)
        
        run_count += epochs
        sess.close()
        return run_count

In [13]:
run_count = 0
run_count = train_model(train_features, train_labels, run_count, 10, 20, save_file)

Initiating the Model


Epoch  1/10: 100%|██████████| 5817/5817 [05:24<00:00, 17.94batches/s]


Epoch: 0001 loss= 0.161737159
Validation Accuracy= 0.937984487 Training Accuracy= 0.966666661


Epoch  2/10:  71%|███████   | 4111/5817 [03:38<01:34, 18.01batches/s]

KeyboardInterrupt: 

In [15]:
# run_count = train_model(run_count, 100, 20, save_file)

In [16]:
save_file = 'model_rgb.ckpt'
saver = tf.train.Saver()

# Launch the graph
with tf.Session() as sess:
    saver.restore(sess, save_file)

    # Check accuracy against Validation data
    validation_accuracy = sess.run(accuracy_operation, feed_dict=valid_feed_dict)
    print('Validation accuracy at {}'.format(validation_accuracy))

    # Check accuracy against Validation data
    test_accuracy = sess.run(accuracy_operation, feed_dict=test_feed_dict)
    print('Test accuracy at {}'.format(test_accuracy))

    exam_size = 500
    y_labels = tf.placeholder(tf.float32, shape=[None, n_classes])
    y_true_cls = tf.argmax(y_labels, 1)
    cls_true, cls_pred = sess.run([y_true_cls, y_pred_cls], feed_dict={features: test_features[0:exam_size], 
                                                                   y_labels: test_labels[0:exam_size]})
    print(cls_true)
    print(cls_pred)

    run_count += epochs
    sess.close()

Validation accuracy at 0.9689922332763672
Test accuracy at 0.9247822761535645
[16  1 38 33 11 38 18 12 25 35 12  7 23  7  4  9 21 20 27 38  4 33  9  3  1
 11 13 10  9 11  5 17 34 23  2 17  3 12 16  8  7 30 18 12 24 25  3 10 18  8
 25 13 15  9 13 35  5 26  9 16 38 10  4  9 15  9 26  2  5 28 11 25 30 34  5
 12  1 10 25 25 21 33 25  7 10 35  3  7 22 13  3  1  2 14 12 32  3 38  9 33
  1 10  5 11 33  4 35 25 33  4  1 14 16 10 30  3 27 29  1 17 13  7  1  8  2
 10 10 30  1  6 36  3 14 13 11 10 18 40  2 38 41  4  6 18 17 25  2 41 11 21
  7 24 11 25 17  3  6  9  7  4 13 16  4 25 18  9 13 14 29 17 13 38 26 25 33
  1  3 40 13  2  8  4 36 25 20 25 18  1 10  8 10 29 12 38 31  2  8 38 18 28
 17  9  4  1 17  9  2 31 13 15 15 38 25  5 25 13 10  5  4 10  2  4  5  1 14
 12 12  5  8 36 25 13 33 18 33 19 12 30  4 18 12 13 20  0 10 40  5  8 12 38
 20 14  0 36 34 28 35 13 25 15 35 14 18 25  1 12  5 25  2 18 18 18 34  9 25
 18 34 39 31  1  9 35 31 26  1  1 33 30 17 13  1 31 13 35  5  1 33 28 35 26
 12  5  2 

NameError: name 'epochs' is not defined

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem?_

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [3]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It would be helpful to plot the images in the notebook._



**Answer:**

In [4]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the dataset?_


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*


**Answer:**

### Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.